In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [143]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/c2_dataset.csv')
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Anho,Ciclo,MCT,ELE,IND,MEC,CIV,CGF,ECA,Recursante,Cant.Materias,Cant.c1,1PC1,2PC1,TC1,NFC1,Cant.c2,Abandono
0,2021,1,0,0,0,0,0,1,0,0,8,6,3,4,5,4,0,0
1,2017,1,0,0,0,0,1,0,0,0,4,1,6,9,3,2,0,1
2,2018,1,0,0,0,0,1,0,0,1,5,1,6,9,3,2,1,1
3,2019,1,0,0,0,0,1,0,0,1,4,1,6,9,3,2,2,1
4,2017,1,0,1,0,0,0,0,0,0,3,1,3,13,10,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2050,2019,1,0,0,0,0,1,0,0,0,5,1,14,34,10,5,0,0
2051,2017,1,0,0,0,0,1,0,0,0,5,1,6,16,2,2,0,0
2052,2022,1,0,0,0,0,1,0,0,0,5,3,17,19,6,2,0,0
2053,2021,2,0,0,1,0,0,0,0,0,6,2,17,15,8,3,0,0


In [144]:
x = df.drop(['Abandono'],axis = 1)
y = df['Abandono']

# Primero probamos con regresion logistica

In [145]:
from sklearn.model_selection import train_test_split
Xtrain,Xtest,Ytrain,Ytest = train_test_split(x,y,test_size=0.2,random_state=30)

In [146]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(Xtrain)
Xscaled = scaler.transform(Xtrain)
XTestscaled = scaler.transform(Xtest)

In [147]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(Xscaled, Ytrain)

LogisticRegression()

In [149]:
predict_log_reg = log_reg.predict(XTestscaled)

In [150]:
from sklearn import metrics
print(metrics.confusion_matrix(Ytest,predict_log_reg))
print(metrics.accuracy_score(Ytest,predict_log_reg))

[[238  42]
 [ 60  71]]
0.7518248175182481


# Ahora probamos Gradient Boosted Trees

In [151]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor


gbrt = GradientBoostingRegressor(max_depth=12, n_estimators=120)
gbrt.fit(Xtrain, Ytrain)
errors = [mean_squared_error(Ytest, y_pred)
for y_pred in gbrt.staged_predict(Xtest)]
bst_n_estimators = np.argmin(errors)

gbrt_best = GradientBoostingRegressor(max_depth=12,n_estimators=bst_n_estimators)
gbrt_best.fit(Xtrain, Ytrain)


GradientBoostingRegressor(max_depth=12, n_estimators=4)

In [152]:
gbrt_predict = gbrt_best.predict(Xtest)
gbrt_predict = (gbrt_predict>=0.5).astype(int)
print(metrics.confusion_matrix(Ytest,gbrt_predict))
print(metrics.accuracy_score(Ytest,gbrt_predict))

[[245  35]
 [ 71  60]]
0.7420924574209246


# Probamos Random Forests

In [153]:
from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=20, n_jobs=-1)
rnd_clf.fit(Xtrain, Ytrain)
rf_predict = rnd_clf.predict(Xtest)

In [154]:
print(metrics.confusion_matrix(Ytest,rf_predict))
print(metrics.accuracy_score(Ytest,rf_predict))

[[243  37]
 [ 56  75]]
0.7737226277372263
